# 用最低的成本将 OpenAI API 切换到智谱AI API

本教程旨在增加帮助大家如何快速，低成本的从 OpenAI 接口迁移到 智谱AI API接口。您可以通过以下两个步骤完成这个工作。

## 1. 更换 API endpoint 或安装 ZhipuAI SDK

这里展示了常见的调用方式，包含了 Function Call, OpenAI API 等功能。您可以通过更换 API endpoint 或安装 ZhipuAI SDK 来完成这个工作。

In [ ]:
from openai import OpenAI
client = OpenAI(api_key="your ZhipuAI api key", base_url="https://open.bigmodel.cn/api/paas/v4")

# from zhipuai import ZhipuAI 
# client = ZhipuAI(api_key="your ZhipuAI api key")  #如果您使用 智谱AI 的SDK，请使用这个代码

# 2. 选择合适的模型

GLM-4系列模型共有四种，分别是 GLM-4-0520,GLM-4-Air,GLM-4-Airx,GLM-4-FLash。对应价格请查看[官网](https://open.bigmodel.cn/pricing)。四种模型具备一定差异：
- GLM-4-0520:  我们当前的最先进最智能的模型，指令遵从能力大幅提升18.6%，具有128k上下文，发布于20240605。 
- GLM-4-V:  支持视觉问答、图像字幕、视觉定位、复杂目标检测等各类图像理解任务，具有128k上下文。 
- GLM-4-Airx:  GLM-4-Air 的高性能版本，效果不变，推理速度达到其2.6倍。具有8k上下文。 
- GLM-4-Air： 性价比最高的版本，综合性能接近GLM-4，具有128k上下文，速度快，价格实惠。 
- GLM-4-Flash： 适用简单任务，速度最快，价格最实惠的版本，具有128k上下文。 

在实际使用的过程中，您需要将要 client设置为对应的模型，例如

In [ ]:
response = client.chat.completions.create(
        model="glm-4-0520"
    )

## 3. 实际演示代码

执行上述两个步骤就已经完成了模型的迁移，在这里，我为大家编写了一个最简单的代码实现，包括了工具调用，常规对话的功能，您可以运行这个代码来直接使用 GLM-4 系列模型执行原本的任务。

In [2]:
from openai import OpenAI
client = OpenAI(api_key="your ZhipuAI api key", base_url="https://open.bigmodel.cn/api/paas/v4")
def function_chat(use_stream=False):
    messages = [
        {
            "role": "user", "content": "What's the Celsius temperature in San Francisco?"
        },
        # Give Observations
        {
            "role": "assistant",
                "content": None,
                "function_call": None,
                "tool_calls": [
                    {
                        "id": "call_1717912616815",
                        "function": {
                            "name": "get_current_weather",
                            "arguments": "{\"location\": \"San Francisco, CA\", \"format\": \"celsius\"}"
                        },
                        "type": "function"
                    }
                ]
        },
        {
            "tool_call_id": "call_1717912616815",
            "role": "tool",
            "name": "get_current_weather",
            "content": "23°C",
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "format": {
                            "type": "string",
                            "enum": ["celsius", "fahrenheit"],
                            "description": "The temperature unit to use. Infer this from the users location.",
                        },
                    },
                    "required": ["location", "format"],
                },
            }
        },
    ]

    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
        tools=tools,
        stream=use_stream,
        max_tokens=256,
        temperature=0.9,
        presence_penalty=1.2,
        top_p=0.1,
        tool_choice="auto"
    )
    if response:
        if use_stream:
            for chunk in response:
                print(chunk)
        else:
            print(response)
    else:
        print("Error:", response.status_code)


def simple_chat(use_stream=False):
    messages = [
        {
            "role": "system",
            "content": "请在你输出的时候都带上“喵喵喵”三个字，放在开头。",
        },
        {
            "role": "user",
            "content": "你是谁"
        }
    ]
    response = client.chat.completions.create(
        model="glm-4-0520",
        messages=messages,
        stream=use_stream,
        max_tokens=256,
        temperature=0.4,
        presence_penalty=1.2,
        top_p=0.8,
    )
    if response:
        if use_stream:
            for chunk in response:
                print(chunk)
        else:
            print(response)
    else:
        print("Error:", response.status_code)


if __name__ == "__main__":
    simple_chat(use_stream=False)
    function_chat(use_stream=False)

ChatCompletion(id='8782707911389443325', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='或者“你是做什么的”这样的问题。\n\n喵喵喵，我是一个人工智能助手，专门设计来回答问题和提供帮助。如果您有任何问题或需要信息，请随时告诉我，我会尽力帮助您。喵喵喵。', role='assistant', function_call=None, tool_calls=None))], created=1719308768, model='glm-4-0520', object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=26, total_tokens=72), request_id='8782707911389443325')
ChatCompletion(id='8782707911389443358', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Celsius temperature in San Francisco is 23°C.', role='assistant', function_call=None, tool_calls=None))], created=1719308769, model='glm-4', object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=203, total_tokens=216), request_id='8782707911389443358')


## 3. 结合开源框架使用

智谱AI API 已经对齐 OpenAI API 标准，使用 GLM-4系列模型能很好的胜任 OpenAI API能制作的部分任务。您也可以参考我们提供的[示例代码](./glm_langchain.ipynb)


如果您使用的是非 LangChain框架，您只需要修改 对应的节点 base_url="https://open.bigmodel.cn/api/paas/v4“ 并修改模型即可。

# 4. 提示词迁移

GLM-4 系列模型相对于 GPT-4 旗舰模型仍然有一定差距。在提示词方面上， GLM-4模型的指令跟随能力能胜任大量常规任务。 您可以在不修改提示词的情况下将完成 GPT2GLM的迁移。

出于稳妥考虑，以下建议可以让模型有更好的表现：
- 提示词中增加 Few-Shot 示例，更多的示例有助于模型理解意图。
- 提示词中多次强掉核心内容，以确保模型能更好的理解。